In [1]:
import pandas as pd
from pickle import dump
pd.set_option("max_rows", None)

Read in race census data. Rename this file when you have a chance.

In [2]:
df_race = pd.read_csv('DECENNIALPL2020.P1_data_with_overlays_2021-10-04T202848.csv', error_bad_lines = False, header = None)

Sort out columns and index.

In [3]:
df_race.columns = df_race.iloc[1]

In [4]:
df_race.drop(df_race.index[1], inplace = True)

In [5]:
 df_race.drop(df_race.index[0], inplace = True)

Get rid of columns detailing multi-race breakdown but keep total of multi-race population.

In [6]:
df_race.drop(df_race.iloc[:, 11:73], axis = 1, inplace = True)

Split county and state into two columns and drop old column.

In [7]:
df_race[['County', 'State']] = df_race['Geographic Area Name'].str.split(',', expand=True)

In [8]:
df_race.drop('Geographic Area Name', axis = 1, inplace = True)

Get rid of punctuation.

In [9]:
df_race.columns = df_race.columns.str.strip().str.replace('[^\w\s]', '')

<ipython-input-9-3176a88a0be0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_race.columns = df_race.columns.str.strip().str.replace('[^\w\s]', '')


In [10]:
def remove_punctuation(x):
    try:
        x = x.str.replace('[^\w\s]','')
    except:
        pass
    return x
df_race = df_race.apply(remove_punctuation)

<ipython-input-10-5daa401cd4a4>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  x = x.str.replace('[^\w\s]','')


In [11]:
df_race.head()

1,id,Total,TotalPopulation of one race,TotalPopulation of one raceWhite alone,TotalPopulation of one raceBlack or African American alone,TotalPopulation of one raceAmerican Indian and Alaska Native alone,TotalPopulation of one raceAsian alone,TotalPopulation of one raceNative Hawaiian and Other Pacific Islander alone,TotalPopulation of one raceSome Other Race alone,TotalPopulation of two or more races,County,State
2,0500000US01001,58805,55648,42160,11445,217,881,35,910,3157,Autauga County,Alabama
3,0500000US01003,231767,216743,189399,18217,1582,2067,143,5335,15024,Baldwin County,Alabama
4,0500000US01005,25223,24523,11317,11933,116,117,1,1039,700,Barbour County,Alabama
5,0500000US01007,22293,21534,16555,4413,60,32,9,465,759,Bibb County,Alabama
6,0500000US01009,59134,55478,50663,845,337,178,24,3431,3656,Blount County,Alabama


Convert race/population info into floats.

In [12]:
df_race[df_race.columns[1:-2]] = df_race[df_race.columns[1:-2]].astype(float)

In [13]:
df_race.head()

1,id,Total,TotalPopulation of one race,TotalPopulation of one raceWhite alone,TotalPopulation of one raceBlack or African American alone,TotalPopulation of one raceAmerican Indian and Alaska Native alone,TotalPopulation of one raceAsian alone,TotalPopulation of one raceNative Hawaiian and Other Pacific Islander alone,TotalPopulation of one raceSome Other Race alone,TotalPopulation of two or more races,County,State
2,0500000US01001,58805.0,55648.0,42160.0,11445.0,217.0,881.0,35.0,910.0,3157.0,Autauga County,Alabama
3,0500000US01003,231767.0,216743.0,189399.0,18217.0,1582.0,2067.0,143.0,5335.0,15024.0,Baldwin County,Alabama
4,0500000US01005,25223.0,24523.0,11317.0,11933.0,116.0,117.0,1.0,1039.0,700.0,Barbour County,Alabama
5,0500000US01007,22293.0,21534.0,16555.0,4413.0,60.0,32.0,9.0,465.0,759.0,Bibb County,Alabama
6,0500000US01009,59134.0,55478.0,50663.0,845.0,337.0,178.0,24.0,3431.0,3656.0,Blount County,Alabama


Remove annoying words.

In [14]:
df_race['County'] = df_race['County'].str.rstrip('County')

In [15]:
df_race['County'] = df_race['County'].str.replace('(Borough|Area|Census|Municipality|and|City|Municipali|ci|city)', '')

<ipython-input-15-d0d98468eeaa>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_race['County'] = df_race['County'].str.replace('(Borough|Area|Census|Municipality|and|City|Municipali|ci|city)', '')


In [16]:
df_race['County'] = df_race['County'].str.strip()

Chugach and Copper River counties in Alaska were combined between the time of the census and the time of the election. Here's my super duper annoying way of handling that...

In [17]:
chugach = (df_race.loc[df_race['County'] == 'Chugach']) 

In [18]:
copper_river = (df_race.loc[df_race['County'] == 'Copper River'])

In [19]:
old_counties = chugach.append(copper_river)
valdez = pd.DataFrame(old_counties.sum(numeric_only = False, axis = 0)).T
df_race.drop(old_counties.index, axis = 0, inplace= True)
df_race = df_race.append(valdez)

In [20]:
valdez

1,id,Total,TotalPopulation of one race,TotalPopulation of one raceWhite alone,TotalPopulation of one raceBlack or African American alone,TotalPopulation of one raceAmerican Indian and Alaska Native alone,TotalPopulation of one raceAsian alone,TotalPopulation of one raceNative Hawaiian and Other Pacific Islander alone,TotalPopulation of one raceSome Other Race alone,TotalPopulation of two or more races,County,State
0,0500000US020630500000US02066,9719.0,8677.0,6518.0,57.0,1332.0,546.0,72.0,152.0,1042.0,ChugachCopper River,Alaska Alaska


In [21]:
df_race = df_race.replace({'0500000US020630500000US02066': 'akcombined', 'ChugachCopper River': 'Valdez Cordova'})
df_race['State'] = df_race['State'].str.replace('Alaska Alaska', 'Alaska')

Drop Puerto Rico because they can not vote in presidential elections.

In [22]:
pr = df_race.loc[df_race['State'].str.contains('Puerto Rico', case=False)]

In [23]:
df_race = df_race.drop(pr.index, axis = 0)

Dropping Kalawoa. Only has 82 residents and does not come up in census pull.

In [24]:
Kalawao = df_race.loc[df_race['County'].str.contains('Kalawao', case=False)]

In [25]:
df_race = df_race.drop(Kalawao.index, axis = 0)

Rename annoying columns.

In [26]:
df_race.rename(columns = {'Total': 'total_pop', 'TotalPopulation of one race': 'total_pop_one_race', 'TotalPopulation of one raceWhite alone':'pop_white', 'TotalPopulation of one raceBlack or African American alone': 'pop_african_american', 'TotalPopulation of one raceAmerican Indian and Alaska Native alone': 'pop_native', 'TotalPopulation of one raceAsian alone': 'pop_asian', 'TotalPopulation of one raceNative Hawaiian and Other Pacific Islander alone': 'pop_islander', 'TotalPopulation of one raceSome Other Race alone': 'pop_other', 'TotalPopulation of two or more races': 'total_pop_two_races'}, inplace = True)

Sort and reset index for joining.

In [27]:
df_race = df_race.sort_values(by = ['State', 'County']).reset_index(drop = True)

In [28]:
df_race.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141 entries, 0 to 3140
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3141 non-null   object 
 1   total_pop             3141 non-null   float64
 2   total_pop_one_race    3141 non-null   float64
 3   pop_white             3141 non-null   float64
 4   pop_african_american  3141 non-null   float64
 5   pop_native            3141 non-null   float64
 6   pop_asian             3141 non-null   float64
 7   pop_islander          3141 non-null   float64
 8   pop_other             3141 non-null   float64
 9   total_pop_two_races   3141 non-null   float64
 10  County                3141 non-null   object 
 11  State                 3141 non-null   object 
dtypes: float64(9), object(3)
memory usage: 294.6+ KB


In [29]:
df_race

1,id,total_pop,total_pop_one_race,pop_white,pop_african_american,pop_native,pop_asian,pop_islander,pop_other,total_pop_two_races,County,State
0,0500000US01001,58805.0,55648.0,42160.0,11445.0,217.0,881.0,35.0,910.0,3157.0,Autauga,Alabama
1,0500000US01003,231767.0,216743.0,189399.0,18217.0,1582.0,2067.0,143.0,5335.0,15024.0,Baldwin,Alabama
2,0500000US01005,25223.0,24523.0,11317.0,11933.0,116.0,117.0,1.0,1039.0,700.0,Barbour,Alabama
3,0500000US01007,22293.0,21534.0,16555.0,4413.0,60.0,32.0,9.0,465.0,759.0,Bibb,Alabama
4,0500000US01009,59134.0,55478.0,50663.0,845.0,337.0,178.0,24.0,3431.0,3656.0,Blount,Alabama
5,0500000US01011,10357.0,10109.0,2320.0,7396.0,72.0,9.0,10.0,302.0,248.0,Bullock,Alabama
6,0500000US01013,19051.0,18464.0,9752.0,8430.0,29.0,143.0,5.0,105.0,587.0,Butler,Alabama
7,0500000US01015,116441.0,110412.0,80586.0,25559.0,487.0,1190.0,119.0,2471.0,6029.0,Calhoun,Alabama
8,0500000US01017,34772.0,33562.0,18850.0,13512.0,106.0,401.0,13.0,680.0,1210.0,Chambers,Alabama
9,0500000US01019,24971.0,24051.0,22707.0,987.0,109.0,56.0,1.0,191.0,920.0,Cherokee,Alabama


In [30]:
dump(df_race, open('df_race1.pkl', 'wb'))